In [ ]:
import pandas as pd
import numpy as np

In [ ]:
diabetes = pd.read_csv(r'/content/drive/MyDrive/train.csv')
diabetes_test = pd.read_csv("/content/drive/MyDrive/test.csv")


In [ ]:
diabetes.head(5)

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [ ]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  700000 non-null  int64  
 1   age                                 700000 non-null  int64  
 2   alcohol_consumption_per_week        700000 non-null  int64  
 3   physical_activity_minutes_per_week  700000 non-null  int64  
 4   diet_score                          700000 non-null  float64
 5   sleep_hours_per_day                 700000 non-null  float64
 6   screen_time_hours_per_day           700000 non-null  float64
 7   bmi                                 700000 non-null  float64
 8   waist_to_hip_ratio                  700000 non-null  float64
 9   systolic_bp                         700000 non-null  int64  
 10  diastolic_bp                        700000 non-null  int64  
 11  heart_rate                

In [ ]:
diabetes.isna().sum()

,0
id,0
age,0
alcohol_consumption_per_week,0
physical_activity_minutes_per_week,0
diet_score,0
sleep_hours_per_day,0
screen_time_hours_per_day,0
bmi,0
waist_to_hip_ratio,0
systolic_bp,0


In [ ]:
diabetes = diabetes.dropna(subset = ["diagnosed_diabetes"])

In [ ]:
x= diabetes.drop(columns = ['diagnosed_diabetes', 'id'])
y = diabetes['diagnosed_diabetes']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
cat_features = x.select_dtypes(include='object').columns.tolist()
num_features = x.select_dtypes(exclude='object').columns.tolist()

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
from catboost import CatBoostClassifier


In [ ]:
x_train_lgb = x_train.copy()
x_val_lgb = x_val.copy()
x_test_lgb = diabetes_test.drop(columns=["id"]).copy()

for col in cat_features:
    le = LabelEncoder()
    combined = pd.concat([
        x_train_lgb[col].astype(str),
        x_val_lgb[col].astype(str),
        x_test_lgb[col].astype(str)
    ])
    le.fit(combined)

    x_train_lgb[col] = le.transform(x_train_lgb[col].astype(str))
    x_val_lgb[col] = le.transform(x_val_lgb[col].astype(str))
    x_test_lgb[col] = le.transform(x_test_lgb[col].astype(str))


In [ ]:
lgb_model = lgb.LGBMClassifier(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary",
    metric="auc",
    random_state=42,
    n_jobs=-1
)

lgb_model.fit(
    x_train_lgb,
    y_train,
    eval_set=[(x_val_lgb, y_val)],
    eval_metric="auc",
    callbacks=[
        lgb.early_stopping(stopping_rounds=200),
        lgb.log_evaluation(200)
    ]
)


[LightGBM] [Info] Number of positive: 349046, number of negative: 210954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.203204 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1648
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623296 -> initscore=0.503564
[LightGBM] [Info] Start training from score 0.503564
Training until validation scores don't improve for 200 rounds
[200]	valid_0's auc: 0.719262
[400]	valid_0's auc: 0.723088
[600]	valid_0's auc: 0.724216
[800]	valid_0's auc: 0.724976
[1000]	valid_0's auc: 0.72545
[1200]	valid_0's auc: 0.725847
[1400]	valid_0's auc: 0.726042
[1600]	valid_0's auc: 0.726131
[1800]	valid_0's auc: 0.726305
[2000]	valid_0's auc: 0.726448
Did not meet early stopping. Best iteration is:
[2000]	valid_0's auc: 0.72

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.03, metric='auc',
               n_estimators=2000, n_jobs=-1, objective='binary',
               random_state=42, subsample=0.8)

In [ ]:
cat_model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=200,
    early_stopping_rounds=200,
    task_type="GPU"
)

cat_model.fit(
    x_train,
    y_train,
    cat_features=cat_features,
    eval_set=(x_val, y_val)
)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6800494	best: 0.6800494 (0)	total: 269ms	remaining: 8m 57s
200:	test: 0.7074048	best: 0.7074048 (200)	total: 35.3s	remaining: 5m 15s
400:	test: 0.7135568	best: 0.7135568 (400)	total: 1m 8s	remaining: 4m 31s
600:	test: 0.7178603	best: 0.7178603 (600)	total: 1m 43s	remaining: 4m 1s
800:	test: 0.7201580	best: 0.7201580 (800)	total: 2m 17s	remaining: 3m 25s
1000:	test: 0.7213090	best: 0.7213090 (1000)	total: 2m 49s	remaining: 2m 48s
1200:	test: 0.7222068	best: 0.7222068 (1200)	total: 3m 18s	remaining: 2m 11s
1400:	test: 0.7228532	best: 0.7228532 (1400)	total: 3m 38s	remaining: 1m 33s
1600:	test: 0.7232533	best: 0.7232544 (1598)	total: 3m 57s	remaining: 59.1s
1800:	test: 0.7235469	best: 0.7235469 (1800)	total: 4m 14s	remaining: 28.2s
1999:	test: 0.7238526	best: 0.7238557 (1998)	total: 4m 35s	remaining: 0us
bestTest = 0.7238556743
bestIteration = 1998
Shrink model to first 1999 iterations.


In [ ]:
cat_test = diabetes_test.drop(columns=["id"]).copy()

for col in cat_features:
    cat_test[col] = cat_test[col].astype(str).fillna("missing")

In [ ]:
pred_cat = cat_model.predict_proba(cat_test)[:, 1]

In [ ]:
pred_lgb = lgb_model.predict_proba(x_test_lgb)[:, 1]

pred_blend_50 = 0.5 * pred_lgb + 0.5 * pred_cat

from scipy.stats import rankdata
pred_rank_avg = (rankdata(pred_lgb) + rankdata(pred_cat)) / 2

In [ ]:
def make_submission(preds, filename):
    submission = pd.DataFrame({
        "id": diabetes_test["id"],
        "diagnosed_diabetes": preds
    })
    submission.to_csv(filename, index=False)
    print(f"Saved {filename} with shape {submission.shape}")

In [ ]:
make_submission(pred_blend_50, "blend_50_50.csv")
make_submission(pred_rank_avg, "rank_avg.csv")

Saved blend_50_50.csv with shape (300000, 2)
Saved rank_avg.csv with shape (300000, 2)


In [ ]:
print(diabetes_test.shape)
print(diabetes_test.head())

(300000, 25)
       id  age  alcohol_consumption_per_week  \
0  700000   45                             4   
1  700001   35                             1   
2  700002   45                             1   
3  700003   55                             2   
4  700004   77                             2   

   physical_activity_minutes_per_week  diet_score  sleep_hours_per_day  \
0                                 100         4.3                  6.8   
1                                  87         3.5                  4.6   
2                                  61         7.6                  6.8   
3                                  81         7.3                  7.3   
4                                  29         7.3                  7.6   

   screen_time_hours_per_day   bmi  waist_to_hip_ratio  systolic_bp  ...  \
0                        6.2  25.5                0.84          123  ...   
1                        9.0  28.6                0.88          120  ...   
2                        

In [ ]:
target = "diagnosed_diabetes"


In [ ]:
x = diabetes.drop(columns=[target, "id"])
y = diabetes[target]

test_x = diabetes_test.drop(columns=["id"])

In [ ]:
cat_features = x.select_dtypes(include="object").columns.tolist()

In [ ]:
x_lgb = x.copy()
test_x_lgb = test_x.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in cat_features:
    le = LabelEncoder()
    combined = pd.concat([x_lgb[col], test_x_lgb[col]]).astype(str)
    le.fit(combined)

    x_lgb[col] = le.transform(x_lgb[col].astype(str))
    test_x_lgb[col] = le.transform(test_x_lgb[col].astype(str))

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

N_FOLDS = 5
skf = StratifiedKFold(
    n_splits=N_FOLDS,
    shuffle=True,
    random_state=42
)

In [ ]:
oof_lgb = np.zeros(len(x))
oof_cat = np.zeros(len(x))

test_preds_lgb = np.zeros((len(test_x), N_FOLDS))
test_preds_cat = np.zeros((len(test_x), N_FOLDS))

In [ ]:
from lightgbm import LGBMClassifier

lgb_params = dict(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=-1,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary",
    metric="auc",
    random_state=42
)

In [ ]:
from catboost import CatBoostClassifier

cat_params = dict(
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=200,
    task_type="GPU"
)


In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(x, y)):
    print(f"\n===== Fold {fold+1}/{N_FOLDS} =====")

    # ---------- SPLITS ----------
    x_train_lgb = x_lgb.iloc[train_idx]
    x_val_lgb   = x_lgb.iloc[val_idx]

    x_train_cat = x.iloc[train_idx]
    x_val_cat   = x.iloc[val_idx]

    y_train = y.iloc[train_idx]
    y_val   = y.iloc[val_idx]

    # ---------- LIGHTGBM ----------
    lgb_model = LGBMClassifier(**lgb_params)
    lgb_model.fit(
        x_train_lgb, y_train,
        eval_set=[(x_val_lgb, y_val)],
        eval_metric="auc"
    )

    oof_lgb[val_idx] = lgb_model.predict_proba(x_val_lgb)[:, 1]
    test_preds_lgb[:, fold] = lgb_model.predict_proba(test_x_lgb)[:, 1]

    # ---------- CATBOOST ----------
    cat_model = CatBoostClassifier(**cat_params)
    cat_model.fit(
        x_train_cat, y_train,
        cat_features=cat_features,
        eval_set=(x_val_cat, y_val),
        use_best_model=True
    )

    oof_cat[val_idx] = cat_model.predict_proba(x_val_cat)[:, 1]
    test_preds_cat[:, fold] = cat_model.predict_proba(test_x)[:, 1]


===== Fold 1/5 =====
[LightGBM] [Info] Number of positive: 349045, number of negative: 210955
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062709 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1640
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623295 -> initscore=0.503556
[LightGBM] [Info] Start training from score 0.503556


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6814423	best: 0.6814423 (0)	total: 76.7ms	remaining: 2m 33s
200:	test: 0.7078753	best: 0.7078753 (200)	total: 11.2s	remaining: 1m 40s
400:	test: 0.7142441	best: 0.7142441 (400)	total: 21.9s	remaining: 1m 27s
600:	test: 0.7188553	best: 0.7188553 (600)	total: 33.3s	remaining: 1m 17s
800:	test: 0.7209570	best: 0.7209570 (800)	total: 44.8s	remaining: 1m 7s
1000:	test: 0.7223849	best: 0.7223849 (1000)	total: 56.4s	remaining: 56.3s
1200:	test: 0.7232943	best: 0.7232943 (1200)	total: 1m 8s	remaining: 45.3s
1400:	test: 0.7238841	best: 0.7238841 (1400)	total: 1m 19s	remaining: 33.8s
1600:	test: 0.7244146	best: 0.7244146 (1600)	total: 1m 30s	remaining: 22.5s
1800:	test: 0.7247968	best: 0.7247968 (1800)	total: 1m 41s	remaining: 11.3s
1999:	test: 0.7250964	best: 0.7250964 (1999)	total: 1m 53s	remaining: 0us
bestTest = 0.7250964344
bestIteration = 1999

===== Fold 2/5 =====
[LightGBM] [Info] Number of positive: 349045, number of negative: 210955
[LightGBM] [Info] Auto-choosing col-wise m

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6785935	best: 0.6785935 (0)	total: 78.7ms	remaining: 2m 37s
200:	test: 0.7059948	best: 0.7059948 (200)	total: 11.5s	remaining: 1m 42s
400:	test: 0.7120838	best: 0.7120838 (400)	total: 22.8s	remaining: 1m 30s
600:	test: 0.7164701	best: 0.7164701 (600)	total: 34.3s	remaining: 1m 19s
800:	test: 0.7189222	best: 0.7189222 (800)	total: 45.3s	remaining: 1m 7s
1000:	test: 0.7200850	best: 0.7200850 (1000)	total: 55.6s	remaining: 55.5s
1200:	test: 0.7209751	best: 0.7209751 (1200)	total: 1m 6s	remaining: 44.4s
1400:	test: 0.7216102	best: 0.7216102 (1400)	total: 1m 18s	remaining: 33.4s
1600:	test: 0.7220150	best: 0.7220150 (1600)	total: 1m 29s	remaining: 22.2s
1800:	test: 0.7223114	best: 0.7223114 (1800)	total: 1m 40s	remaining: 11.1s
1999:	test: 0.7226194	best: 0.7226257 (1995)	total: 1m 50s	remaining: 0us
bestTest = 0.7226257324
bestIteration = 1995
Shrink model to first 1996 iterations.

===== Fold 3/5 =====
[LightGBM] [Info] Number of positive: 349046, number of negative: 210954
[Li

...

0:	test: 0.6803592	best: 0.6803592 (0)	total: 117ms	remaining: 3m 53s
200:	test: 0.7060333	best: 0.7060333 (200)	total: 23.8s	remaining: 3m 33s
400:	test: 0.7126992	best: 0.7126992 (400)	total: 44.7s	remaining: 2m 58s
600:	test: 0.7173671	best: 0.7173671 (600)	total: 1m 9s	remaining: 2m 41s
800:	test: 0.7197715	best: 0.7197715 (800)	total: 1m 33s	remaining: 2m 20s
1000:	test: 0.7210497	best: 0.7210497 (1000)	total: 1m 57s	remaining: 1m 57s
1200:	test: 0.7220834	best: 0.7220834 (1200)	total: 2m 21s	remaining: 1m 34s
1400:	test: 0.7226842	best: 0.7226842 (1400)	total: 2m 45s	remaining: 1m 10s
1600:	test: 0.7231190	best: 0.7231190 (1600)	total: 3m 9s	remaining: 47.1s
1800:	test: 0.7235605	best: 0.7235612 (1790)	total: 3m 35s	remaining: 23.8s
1999:	test: 0.7239551	best: 0.7239552 (1995)	total: 4m	remaining: 0us
bestTest = 0.7239551544
bestIteration = 1995
Shrink model to first 1996 iterations.

===== Fold 4/5 =====
[LightGBM] [Info] Number of positive: 349046, number of negative: 210954
[L

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6798114	best: 0.6798114 (0)	total: 92.7ms	remaining: 3m 5s
200:	test: 0.7072468	best: 0.7072468 (200)	total: 16.5s	remaining: 2m 27s
400:	test: 0.7137680	best: 0.7137680 (400)	total: 32.7s	remaining: 2m 10s
600:	test: 0.7186200	best: 0.7186200 (600)	total: 50.5s	remaining: 1m 57s
800:	test: 0.7209530	best: 0.7209530 (800)	total: 1m 8s	remaining: 1m 42s
1000:	test: 0.7221786	best: 0.7221786 (1000)	total: 1m 24s	remaining: 1m 24s
1200:	test: 0.7230484	best: 0.7230484 (1200)	total: 1m 41s	remaining: 1m 7s
1400:	test: 0.7236092	best: 0.7236092 (1400)	total: 1m 58s	remaining: 50.7s
1600:	test: 0.7241623	best: 0.7241623 (1600)	total: 2m 20s	remaining: 35s
1800:	test: 0.7246633	best: 0.7246633 (1800)	total: 2m 46s	remaining: 18.4s
1999:	test: 0.7249610	best: 0.7249610 (1999)	total: 3m 15s	remaining: 0us
bestTest = 0.7249609828
bestIteration = 1999

===== Fold 5/5 =====
[LightGBM] [Info] Number of positive: 349046, number of negative: 210954
[LightGBM] [Info] Auto-choosing row-wise 

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6777874	best: 0.6777874 (0)	total: 90.5ms	remaining: 3m
200:	test: 0.7073883	best: 0.7073883 (200)	total: 16.7s	remaining: 2m 29s
400:	test: 0.7137893	best: 0.7137893 (400)	total: 34.3s	remaining: 2m 16s
600:	test: 0.7180615	best: 0.7180615 (600)	total: 51.1s	remaining: 1m 59s
800:	test: 0.7205803	best: 0.7205803 (800)	total: 1m 8s	remaining: 1m 43s
1000:	test: 0.7218547	best: 0.7218547 (1000)	total: 1m 26s	remaining: 1m 26s
1200:	test: 0.7227958	best: 0.7227958 (1200)	total: 1m 42s	remaining: 1m 8s
1400:	test: 0.7234968	best: 0.7234968 (1400)	total: 1m 59s	remaining: 51.2s
1600:	test: 0.7240009	best: 0.7240009 (1600)	total: 2m 16s	remaining: 34s
1800:	test: 0.7243416	best: 0.7243416 (1800)	total: 2m 38s	remaining: 17.5s
1999:	test: 0.7246172	best: 0.7246172 (1995)	total: 3m 5s	remaining: 0us
bestTest = 0.7246171832
bestIteration = 1995
Shrink model to first 1996 iterations.
